#1. Connexion au drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# 2. Fonction extraction des audios

In [ ]:
#################################################################################################################################
# Fonction qui permet d'extraire l'audio d'un fichier mp4
# *Parameters : 
#   - num_fichier_zip : numéro présent au début du nom des fichiers (ex : 9-20201023T121602Z-001.zip --> num_fichier_zip = 9)
#################################################################################################################################
def extract_audio(num_fichier_zip):
    # On ouvre le fichier zip en mode lecture

    for i in liste_path: #On parcours les fichiers zip du même nom
        with ZipFile(i, 'r') as zip:
            # On affiche le contenu du fichier zip
            zip.printdir()

            # Extraction des fichiers
            print('Extracting all the files now...')
            zip.extractall()
            print('Done!')

    
    dic={} #Définition du dictionnaire
    chemin = '/content/'+str(num_fichier_zip)+'/' #Chemin où sont stockés les fichiers mp4
    List = os.listdir('/content/'+str(num_fichier_zip)+'/') #Liste de tous les fichiers zip
    print(List)
    #On parcours l'ensemble des fichiers mp4 présent dans la liste
    for filename in List: 
        nom_fichier = filename[:-4] #On récupère l'id de la video (ex : 1234.mp4 --> nom fichier = 1234)
        input = str(chemin+filename) #En input on a le chemin du fichier avec son nom
        print(input)
        output = str('audio_'+str(num_fichier_zip)+'_'+str(nom_fichier)+'.wav') #On souhaite avoir en sortie un audio .wav
        print(output)

        #Fonction unix adaptée à python qui permet d'extraire l'audio d'une video
        #Fonction de la librairie ffmpeg
        #8000 Hz, 1 canal --> cela permet de diminuer la taille du fichier final
        subprocess.call(['ffmpeg', '-i', input, '-ar', '8000', '-ac', '1', '-f', 'wav', '-ab', '192k', '-vn', output])
        audio, sr = librosa.load(output, sr=8000) #On recupère le signal avec librosa.load
        dic[nom_fichier]=audio #On complète le dictionnaire avec le nom du fichier (id) et le signal
    return dic

In [ ]:
#import
!pip install ffmpeg-python
from zipfile import ZipFile
import numpy as np
import pandas as pd
import os
import ffmpeg
import subprocess
import librosa

#chemin principal
path = '/content/drive/MyDrive/Projet Non-Alternants AED 2021/DATA/VIDEOS/'

#liste des chemins pour le fichiers zip
# liste_path = [path + '9-20201023T121602Z-001.zip', path + '9-20201023T121602Z-002.zip']
# liste_path = [path + '8-20201023T120528Z-001.zip', path + '8-20201023T120528Z-002.zip', path + '8-20201023T120528Z-003.zip', path + '8-20201023T120528Z-004.zip']
# liste_path = [path + '7-20201023T115817Z-001.zip', path + '7-20201023T115817Z-002.zip', path + '7-20201023T115817Z-003.zip', path + '7-20201023T115817Z-004.zip']
# liste_path = [path + '6-20201023T114823Z-001.zip', path + '6-20201023T114823Z-002.zip', path + '6-20201023T114823Z-003.zip', path + '6-20201023T114823Z-004.zip']
# liste_path = [path + '5-20201023T105216Z-001.zip', path + '5-20201023T105216Z-002.zip', path + '5-20201023T105216Z-003.zip', path + '5-20201023T105216Z-004.zip', path + '5-20201023T105216Z-005.zip']
# liste_path = [path + '4-20201023T103633Z-001.zip', path + '4-20201023T103633Z-002.zip', path + '4-20201023T103633Z-003.zip', path + '4-20201023T103633Z-004.zip', path + '4-20201023T103633Z-005.zip']
# liste_path = [path + '3-20201023T102506Z-001.zip', path + '3-20201023T102506Z-002.zip', path + '3-20201023T102506Z-003.zip', path + '3-20201023T102506Z-004.zip']
# liste_path = [path + '2-20201023T101143Z-001.zip', path + '2-20201023T101143Z-002.zip', path + '2-20201023T101143Z-003.zip', path + '2-20201023T101143Z-004.zip', path + '2-20201023T101143Z-005.zip']
liste_path = [path + '1-20201023T100155Z-001.zip', path + '1-20201023T100155Z-002.zip', path + '1-20201023T100155Z-003.zip', path + '1-20201023T100155Z-004.zip']


dic=extract_audio(1) #On crée le dictionnaire avec la fonction extract_audio

File Name                                             Modified             Size
1/117867.mp4                                   2020-10-11 00:22:16      3895977
1/118898.mp4                                   2020-10-11 00:22:24      4008398
1/119141.mp4                                   2020-10-11 00:22:32      5422223
1/118700.mp4                                   2020-10-11 00:22:22      9145859
1/120466.mp4                                   2020-10-11 00:22:36      8040263
1/117590.mp4                                   2020-10-11 00:22:12      6900830
1/117192.mp4                                   2020-10-11 00:22:10      2650400
1/117123.mp4                                   2020-10-11 00:22:06      9505558
1/117887.mp4                                   2020-10-11 00:22:20      7200951
1/117533.mp4                                   2020-10-11 00:22:10      3804854
1/116797.mp4                                   2020-10-13 01:25:20      7974334
1/118930.mp4                            

# 3. On exporte l'id des films ainsi que le signal correspondant dans un fichier pickle

In [ ]:
# Le dictionnaire contenant l'identifiant de la vidéo et le signal étant très volumineux, nous avons choisi
# de le stocker sous forme d'un fichier .pickle
import pickle

with open('/content/drive/MyDrive/Projet Non-Alternants AED 2021/DATA/AUDIO/audio_1.pickle', 'wb') as handle:
    pickle.dump(dic, handle, protocol=pickle.HIGHEST_PROTOCOL)

# 4. Features

**Automatisation**

In [ ]:
#Import
from zipfile import ZipFile
import numpy as np
import pandas as pd
import librosa
from tqdm import tqdm

In [ ]:
#################################################################################################################################
# Fonction transform_pickle_df, permet de transformer le fichier pickle en dataframe avec :
# - en index les identifiants des vidéos
# - une colonne signal contenant le signal de l'audio
# *Parameters : 
#   - num_audio : numéro du nom de fichier de l'audio (ex : audio_1.zip --> num_audio = 1)
#################################################################################################################################
def transform_pickle_df(num_audio):

    #On dézip le fichier pickle
    with ZipFile('/content/drive/MyDrive/Projet Non-Alternants AED 2021/DATA/AUDIO/audio_' +str(num_audio) + '.zip', 'r') as zip:
        # On affiche les fichiers présent dans le dossier
        zip.printdir()

        # Extraction des fichiers
        print('Extracting all the files now...')
        zip.extractall()
        print('Done!')

    #On transforme le fichier pickle en dataframe
    df = pd.read_pickle('/content/audio_' + str(num_audio) + '.pickle')
    #On donne un nom à la colonne contenant le signal
    #On transpose le dataframe car il n'était pas dans le bon sens
    df_audio = pd.DataFrame([df], index=['signal']).T 
    return df_audio

In [ ]:
#################################################################################################################################
# Source : https://larevueia.fr/machine-learning-pour-la-classification-automatique-de-musiques-avec-python/
# Fonction audio_pipeline permet de créer l'ensemble des features pour un signal donnée
# *Parameters :
#   - audio : signal (array de x chiffre représentant le signal de l'audio)
#################################################################################################################################
def audio_pipeline(audio):

    features = [] #initilisationd de la liste contenant les features

    #Calcul de la durée (en secondes)
    #la durée correspond à la longueur du signal / 8000 (on a 8000 valeurs par secondes)
    duree = len(audio)/8000
    features.append(duree)

    # Calcul du ZCR
    # ZCR est un indicateur proportionnel au taux de changement de signe d’un signal musical
    # Plus concrètement, on va savoir le nombre de fois que le signal coupe l'axe des abscisses
    zcr = librosa.zero_crossings(audio)
    features.append(sum(zcr))

    # Calcul de la moyenne du Spectral centroid
    # Le spectral centroid est une métrique utilisée pour caractériser les spectres des musiques. 
    # Il s’agit d’un indicateur géométrique de la position du centre de masse du spectre d’un son
    spectral_centroids = librosa.feature.spectral_centroid(audio)
    features.append(np.mean(spectral_centroids))
    
    # Calcul du spectral rolloff point
    # Mesure de l’asymétrie à droite d’un spectre sonore. 
    # Elle est utilisée par exemple pour différencier des paroles chantées et des paroles normales
    rolloff = librosa.feature.spectral_rolloff(audio)
    features.append(np.mean(rolloff))

    # Calcul des moyennes des MFCC
    # Les MFCC sont des coefficients cepstraux qui correspondent à une transformation sinusoïdale de la puissance d’un signal
    # - Calcul de la transformée de Fourier de la trame à analyser
    # - Pondération du spectre d’amplitude (ou de puissance selon les cas) par un banc de filtres triangulaires espacés selon l’échelle de Mel
    # - Calcul de la transformée en cosinus discrète du log-mel-spectre
    mfcc = librosa.feature.mfcc(audio,n_mfcc=13) #On a choisi de prendre les 13 premiers coefficients mfcc

    # On fait la moyenne des coefficients mfcc
    for x in mfcc:
        features.append(np.mean(x))

    return features

In [ ]:
#################################################################################################################################
# Fonction dataframe_complet permet de créer le dataframe contenant l'ensemble des features
# *Parameters : 
#   - numero_audio : numéro du nom de fichier de l'audio (ex : audio_1.zip --> num_audio = 1)
#################################################################################################################################
def dataframe_complet(numero_audio):
    # Fonction transform_pickle_df pour transformer le dictionnaire en dataframe
    df = transform_pickle_df(numero_audio)
    # Nom des features
    columns_name = ['duree', 'zrc', 'spectral_c', 'rolloff', 'mfcc1', 
                'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 
                'mfcc9','mfcc10', 'mfcc11', 'mfcc12', 'mfcc13']

    # On crée un dataframe vide avec le nom des colonnes
    df_final = pd.DataFrame(columns=columns_name)
    # audio_files contient l'ensemble des signaux 
    audio_files = df['signal']

    # Pour chaque signal on applique la fonction audio_pipeline qui crée les features
    for s in tqdm(range(len(audio_files))):
        df_final.loc[s] = audio_pipeline(audio_files[s])

    # On crée la colonne id avec l'index du dataframe créé à la base du dictionnaire
    df_final['id'] = df.index
    # On sauvegarde le dataframe en format csv
    df_final.to_csv('/content/df_'+str(numero_audio)+'.csv', index = False)

In [ ]:
dataframe_complet(9)

File Name                                             Modified             Size
audio_9.pickle                                 2021-11-03 09:30:10   1253058170
Extracting all the files now...
Done!


100%|██████████| 306/306 [13:08<00:00,  2.58s/it]


In [ ]:
df_final

,duree,zrc,spectral_c,rolloff,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,id
0,148.607750,226345.0,3085.495886,5986.610685,-98.231995,37.471394,-42.313953,-23.037893,-35.917725,-21.154766,-25.531017,-17.526449,-18.862604,-12.678902,-15.215623,-6.590576,-6.783550,1341
1,101.703500,165611.0,3441.622458,7129.196639,-34.139572,54.185520,-0.436810,4.506167,-7.122158,0.009030,-6.883800,-8.948134,-13.295561,-6.596420,-9.484797,-2.844001,-7.826204,1405
2,142.175875,96657.0,1992.086251,4287.249113,-146.228333,92.757355,5.222880,26.853729,5.794054,10.664813,2.643010,3.774183,-0.647875,0.890524,-4.156791,1.616906,-4.355479,105954
3,147.261000,183169.0,2985.427190,6160.049554,-137.687851,67.028183,-9.572527,-1.992519,-9.117128,-0.648659,-7.531102,-4.302991,-13.316188,-5.421843,-4.864212,-3.016084,-6.592301,102905
4,90.720375,84527.0,2274.858012,4695.901106,-130.235809,82.808762,-3.236480,12.612826,-2.946570,9.786930,-2.886461,0.509199,-6.269305,2.207209,-1.120280,0.000745,-8.284515,114935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,30.441375,58021.0,3484.572252,6541.999727,-187.915298,36.009480,-36.957115,-5.397773,-22.798347,-11.139829,-19.772490,-8.905482,-13.724164,-3.714729,-7.570379,-2.626836,-4.892546,1388
1004,76.834875,90303.0,2962.649346,6174.713231,-226.904770,67.629051,-9.912333,-1.631637,-9.149450,6.952187,-10.231359,-8.324279,-7.089167,1.684935,-9.238454,0.008538,-9.195056,112450
1005,94.087250,201681.0,4195.496824,7689.117452,-136.844086,14.627162,-10.932882,-11.399196,-16.436459,-1.946808,-2.467423,-2.126757,-4.651204,3.489320,1.008218,4.544727,-1.852265,1955
1006,60.023625,116591.0,3621.014049,7052.066632,-154.221786,36.443451,-28.561970,-6.128111,-10.352930,-2.543650,-11.101056,-4.242270,-9.466120,-4.497554,-9.247684,-0.960680,-6.940629,1793


#5. Concaténation des dataframes

In [ ]:
#Import
import pandas as pd

In [ ]:
#Lecture des dataframes
liste_df = []
for i in range(0,9):
    df = pd.read_csv('/content/drive/MyDrive/Projet Non-Alternants AED 2021/DATA/AUDIO/df_'+str(i+1)+'.csv')
    liste_df.append(df)

In [ ]:
#concaténation des dataframes
df_audio = pd.concat([liste_df[0],liste_df[1],liste_df[2],liste_df[3],liste_df[4],liste_df[5],liste_df[6],liste_df[7],liste_df[8]])

In [ ]:
df_audio

,duree,zrc,spectral_c,rolloff,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,id
0,148.607750,226345.0,3085.495886,5986.610685,-98.231995,37.471394,-42.313953,-23.037893,-35.917725,-21.154766,-25.531017,-17.526449,-18.862604,-12.678902,-15.215623,-6.590576,-6.783550,1341
1,101.703500,165611.0,3441.622458,7129.196639,-34.139572,54.185520,-0.436810,4.506167,-7.122158,0.009030,-6.883800,-8.948134,-13.295561,-6.596420,-9.484797,-2.844001,-7.826204,1405
2,142.175875,96657.0,1992.086251,4287.249113,-146.228333,92.757355,5.222880,26.853729,5.794054,10.664813,2.643010,3.774183,-0.647875,0.890524,-4.156791,1.616906,-4.355479,105954
3,147.261000,183169.0,2985.427190,6160.049554,-137.687851,67.028183,-9.572527,-1.992519,-9.117128,-0.648659,-7.531102,-4.302991,-13.316188,-5.421843,-4.864212,-3.016084,-6.592301,102905
4,90.720375,84527.0,2274.858012,4695.901106,-130.235809,82.808762,-3.236480,12.612826,-2.946570,9.786930,-2.886461,0.509199,-6.269305,2.207209,-1.120280,0.000745,-8.284515,114935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7811,150.209875,157853.0,2812.053254,5992.452904,-154.165543,72.064880,-0.470686,13.066220,-0.227326,5.366939,-7.881814,-3.382499,-8.755445,-4.256695,-10.343851,-4.562756,-6.817455,90524
7812,93.785375,83477.0,2477.724820,5454.730499,-246.726685,75.601212,-1.169587,-23.782955,-26.809723,-23.023336,-22.503626,-5.569940,-9.317266,-6.321781,-3.210515,-1.434442,-8.404831,96020
7813,107.229750,88943.0,2186.030370,4437.972607,-229.962463,90.738815,6.287278,20.709890,-1.125464,6.946919,-10.411656,-2.948574,-9.062596,-2.072078,-2.163980,-0.825338,-7.647690,97701
7814,89.605875,157869.0,3348.646546,6069.827251,-173.058304,30.449316,-48.013508,-8.267272,-31.391747,-8.551555,-22.810020,-14.763551,-16.354713,-11.663852,-13.973055,-4.326704,-11.226606,905


#6. Ajout des ratings

In [ ]:
#On import le dataframe texte
!gdown --id 1QS4KxTpqnwEJT3EisRh89Hg1AFgcJlhY
!unzip -qq challengeSIDM2-movies.min30.withtext.zip

Downloading...
From: https://drive.google.com/uc?id=1QS4KxTpqnwEJT3EisRh89Hg1AFgcJlhY
To: /content/challengeSIDM2-movies.min30.withtext.zip
100% 18.1M/18.1M [00:00<00:00, 108MB/s] 


In [ ]:
#Lecture du dataframe texte
df = pd.read_csv('challengeSIDM2-movies.min30.withtext.csv')
df.head()

In [ ]:
#On recupère l'identifiant des films ainsi que le rating dans un dataframe nommé rating
rating=df[['movieId','rating']]
df_audio['id']=df_audio['id'].astype(int)

In [ ]:
#On joint le dataframe contenant les features et le dataframe contenant les ratings à l'aide de la clé des identifiants
df_audio_final = pd.merge(df_audio, rating, left_on = ['id'], right_on = ['movieId'])

In [ ]:
#On supprime movieId car on a deux colonnes ID
del df_audio_final['movieId']

In [ ]:
#Sauvegarde du dataframe complet en csv
df_audio_final.to_csv('/content/df_audio.csv', index = False)

In [ ]:
#Lecture du fichier csv
df_audio = pd.read_csv('/content/drive/MyDrive/Projet Non-Alternants AED 2021/DATA/AUDIO/df_audio.csv')

In [ ]:
df_audio

,duree,zrc,spectral_c,rolloff,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,id,rating
0,148.607750,226345.0,3085.495886,5986.610685,-98.231995,37.471394,-42.313953,-23.037893,-35.917725,-21.154766,-25.531017,-17.526449,-18.862604,-12.678902,-15.215623,-6.590576,-6.783550,1341,2.866841
1,101.703500,165611.0,3441.622458,7129.196639,-34.139572,54.185520,-0.436810,4.506167,-7.122158,0.009030,-6.883800,-8.948134,-13.295561,-6.596420,-9.484797,-2.844001,-7.826204,1405,2.967150
2,142.175875,96657.0,1992.086251,4287.249113,-146.228333,92.757355,5.222880,26.853729,5.794054,10.664813,2.643010,3.774183,-0.647875,0.890524,-4.156791,1.616906,-4.355479,105954,3.494819
3,147.261000,183169.0,2985.427190,6160.049554,-137.687851,67.028183,-9.572527,-1.992519,-9.117128,-0.648659,-7.531102,-4.302991,-13.316188,-5.421843,-4.864212,-3.016084,-6.592301,102905,3.110000
4,90.720375,84527.0,2274.858012,4695.901106,-130.235809,82.808762,-3.236480,12.612826,-2.946570,9.786930,-2.886461,0.509199,-6.269305,2.207209,-1.120280,0.000745,-8.284515,114935,3.657233
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7792,150.209875,157853.0,2812.053254,5992.452904,-154.165543,72.064880,-0.470686,13.066220,-0.227326,5.366939,-7.881814,-3.382499,-8.755445,-4.256695,-10.343851,-4.562756,-6.817455,90524,2.919580
7793,93.785375,83477.0,2477.724820,5454.730499,-246.726685,75.601212,-1.169587,-23.782955,-26.809723,-23.023336,-22.503626,-5.569940,-9.317266,-6.321781,-3.210515,-1.434442,-8.404831,96020,3.986486
7794,107.229750,88943.0,2186.030370,4437.972607,-229.962463,90.738815,6.287278,20.709890,-1.125464,6.946919,-10.411656,-2.948574,-9.062596,-2.072078,-2.163980,-0.825338,-7.647690,97701,2.391026
7795,89.605875,157869.0,3348.646546,6069.827251,-173.058304,30.449316,-48.013508,-8.267272,-31.391747,-8.551555,-22.810020,-14.763551,-16.354713,-11.663852,-13.973055,-4.326704,-11.226606,905,4.170053
